In [ ]:
from IPython import get_ipython
from IPython.display import display

!pip install pandas requests beautifulsoup4 pdfminer.six lxml > /dev/null 2>&1

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ----------SOLUTION REUSE SYSTEM - TAHAP 4 (CORRECTED)-------------

import pandas as pd
import numpy as np
import json
import os
from datetime import date
import re
from collections import Counter
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# Load required models and data - CORRECTED PATHS
def load_system_components():
    """Load all required components for solution reuse"""
    try:
        # Load processed DRUG CRIME cases - CORRECTED
        processed_folder = "/content/drive/MyDrive/data/processed"
        cases_path = os.path.join(processed_folder, "drug_crime_cases_with_classification.csv")  # CORRECTED
        df = pd.read_csv(cases_path)

        # Load DRUG CRIME models - CORRECTED
        models_dir = "/content/drive/MyDrive/models"
        tfidf_vectorizer = joblib.load(os.path.join(models_dir, 'drug_tfidf_vectorizer.pkl'))  # CORRECTED
        tfidf_matrix = np.load(os.path.join(models_dir, 'drug_tfidf_matrix.npy'))  # CORRECTED

        print(f"✓ Loaded {len(df)} drug crime cases")
        print(f"✓ Loaded TF-IDF vectorizer with {tfidf_matrix.shape[1]} features")

        return df, tfidf_vectorizer, tfidf_matrix
    except Exception as e:
        print(f"Error loading components: {e}")
        return None, None, None

# Load system components
df, tfidf_vectorizer, tfidf_matrix = load_system_components()

if df is None:
    print("Failed to load system components. Please run previous stages first.")
    exit()

# =====================================================
# i. EKSTRAKSI SOLUSI DARI KASUS LAMA - CORRECTED FOR DRUG CRIMES
# =====================================================

def extract_solution_from_case(case_data):
    """
    Extract solution components from a drug crime case

    Args:
        case_data: Row from dataframe containing case information

    Returns:
        dict: Solution components including amar, punishment, legal basis
    """
    solution = {
        'case_id': case_data.get('case_id', ''),
        'amar_putusan': '',
        'jenis_hukuman': '',
        'durasi_hukuman': '',
        'jumlah_denda': '',
        'pasal_terkait': '',
        'dasar_hukum': '',
        'pertimbangan_hakim': '',
        'ringkasan_solusi': '',
        # ADDED: Drug crime specific fields
        'jenis_narkoba': '',
        'jenis_tindakan': '',
        'status_putusan': ''
    }

    # Extract amar putusan (court decision)
    if pd.notna(case_data.get('amar')):
        solution['amar_putusan'] = str(case_data['amar'])[:500]  # Limit length

    # Extract punishment details - CORRECTED for drug crimes
    if pd.notna(case_data.get('jenis_hukuman')):
        solution['jenis_hukuman'] = str(case_data['jenis_hukuman'])
    elif pd.notna(case_data.get('status_putusan')):
        solution['jenis_hukuman'] = str(case_data['status_putusan'])

    if pd.notna(case_data.get('durasi_hukuman')):
        solution['durasi_hukuman'] = str(case_data['durasi_hukuman'])

    if pd.notna(case_data.get('jumlah_denda')):
        solution['jumlah_denda'] = str(case_data['jumlah_denda'])

    # Extract legal articles - CORRECTED for drug crimes
    if pd.notna(case_data.get('pasal')):
        solution['pasal_terkait'] = str(case_data['pasal'])
    elif pd.notna(case_data.get('pasal_terkait')):
        solution['pasal_terkait'] = str(case_data['pasal_terkait'])

    # ADDED: Drug crime specific fields
    if pd.notna(case_data.get('jenis_narkoba')):
        solution['jenis_narkoba'] = str(case_data['jenis_narkoba'])
    elif pd.notna(case_data.get('klasifikasi_narkoba')):
        solution['jenis_narkoba'] = str(case_data['klasifikasi_narkoba'])

    if pd.notna(case_data.get('jenis_tindakan')):
        solution['jenis_tindakan'] = str(case_data['jenis_tindakan'])

    if pd.notna(case_data.get('status_putusan')):
        solution['status_putusan'] = str(case_data['status_putusan'])

    # Extract legal basis from text - CORRECTED to use drug crime text columns
    full_text = ""
    for col in ['text_full', 'ringkasan_fakta', 'argumen_hukum']:
        if col in case_data and pd.notna(case_data.get(col)):
            full_text += str(case_data[col]) + " "

    legal_basis = extract_legal_basis(full_text)
    solution['dasar_hukum'] = legal_basis

    # Extract judge considerations
    considerations = extract_judge_considerations(full_text)
    solution['pertimbangan_hakim'] = considerations

    # Create solution summary - ENHANCED for drug crimes
    solution['ringkasan_solusi'] = create_solution_summary(solution)

    return solution

def extract_legal_basis(text):
    """Extract legal basis from case text - ENHANCED for drug crimes"""
    if pd.isna(text) or not text.strip():
        return ""

    text = str(text).lower()

    # Look for legal basis patterns - ENHANCED for drug crimes
    legal_patterns = [
        r'undang-undang.*?nomor.*?35.*?tahun.*?2009',  # UU Narkotika
        r'uu.*?35.*?2009',
        r'undang-undang.*?narkotika',
        r'berdasarkan.*?pasal.*?(\d+)',
        r'undang-undang.*?nomor.*?(\d+)',
        r'kitab.*?hukum.*?pidana',
        r'peraturan.*?pemerintah',
        r'pasal.*?(\d+).*?uu.*?35'
    ]

    legal_basis = []
    for pattern in legal_patterns:
        matches = re.findall(pattern, text)
        if matches:
            legal_basis.extend([str(m) for m in matches[:2]])  # Limit matches

    return '; '.join(set(legal_basis))

def extract_judge_considerations(text):
    """Extract judge considerations from case text"""
    if pd.isna(text) or not text.strip():
        return ""

    text = str(text).lower()

    # Look for consideration patterns
    consideration_patterns = [
        r'menimbang.*?bahwa.*?[.;]',
        r'mempertimbangkan.*?[.;]',
        r'dengan.*?pertimbangan.*?[.;]',
        r'mengingat.*?[.;]',
        r'berdasarkan.*?fakta.*?[.;]'
    ]

    considerations = []
    for pattern in consideration_patterns:
        matches = re.findall(pattern, text)
        considerations.extend(matches[:2])  # Limit matches

    return ' '.join(considerations)[:300]  # Limit length

def create_solution_summary(solution):
    """Create a summary of the solution - ENHANCED for drug crimes"""
    summary_parts = []

    # ADDED: Drug crime specific info
    if solution['jenis_narkoba']:
        summary_parts.append(f"Jenis Narkoba: {solution['jenis_narkoba']}")

    if solution['jenis_tindakan']:
        summary_parts.append(f"Tindakan: {solution['jenis_tindakan']}")

    if solution['jenis_hukuman']:
        summary_parts.append(f"Jenis hukuman: {solution['jenis_hukuman']}")

    if solution['durasi_hukuman']:
        summary_parts.append(f"Durasi: {solution['durasi_hukuman']}")

    if solution['jumlah_denda']:
        summary_parts.append(f"Denda: {solution['jumlah_denda']}")

    if solution['pasal_terkait']:
        summary_parts.append(f"Pasal: {solution['pasal_terkait']}")

    if solution['status_putusan']:
        summary_parts.append(f"Status: {solution['status_putusan']}")

    return '; '.join(summary_parts)

# =====================================================
# ii. ALGORITMA PREDIKSI - CORRECTED
# =====================================================

def predict_outcome(query: str) -> str:
    """
    Predict case outcome based on similar cases

    Args:
        query (str): New case description

    Returns:
        str: Predicted outcome/solution
    """
    # Step 1: Get top-k similar cases using CORRECTED retrieve function
    top_k = retrieve(query, k=5)

    # Step 2: Extract solutions from similar cases
    solutions = [case_solutions[c] for c in top_k]

    # Step 3: Apply prediction algorithms

    # Majority Vote
    majority_solution = apply_majority_vote(solutions)

    # Weighted Similarity
    weighted_solution = apply_weighted_similarity(query, top_k, solutions)

    # Combine predictions
    final_prediction = combine_predictions(majority_solution, weighted_solution)

    return final_prediction

def retrieve(query: str, k: int = 5) -> list:
    """
    Retrieve top-k most similar drug crime cases for a given query
    CORRECTED to match case retrieval system
    """
    # CORRECTED: Use the same text cleaning as in case retrieval
    def clean_text_for_drug_cases(text):
        """Clean text for TF-IDF processing - enhanced for drug cases"""
        if pd.isna(text):
            return ""

        text = str(text).lower()
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text)

        # CORRECTED: Use same stopwords as case retrieval
        stopwords = ['dan', 'atau', 'yang', 'untuk', 'dari', 'dengan', 'pada',
                    'dalam', 'ke', 'di', 'adalah', 'oleh', 'akan', 'telah', 'sudah',
                    'ini', 'itu', 'tidak', 'ada', 'juga', 'dapat', 'bisa', 'harus',
                    'maka', 'saja', 'hanya', 'masih', 'sebuah', 'satu', 'dua',
                    'bahwa', 'tersebut', 'sebagai', 'atas', 'karena', 'sehingga']

        # CORRECTED: Include drug terms
        drug_terms = ['uud', 'kuhp', 'uu', 'pp', 'thc', 'mdma']
        words = text.split()
        words = [word for word in words if (len(word) > 2 or word in drug_terms) and word not in stopwords]
        return ' '.join(words)

    processed_query = clean_text_for_drug_cases(query)
    query_vector = tfidf_vectorizer.transform([processed_query])

    # Calculate similarities
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Get top-k indices
    top_k_indices = similarities.argsort()[-k:][::-1]

    # Return case IDs - CORRECTED to match case retrieval format
    case_ids = []
    for idx in top_k_indices:
        case_id = df.iloc[idx].get('case_id', f"drug_case_{idx+1:03d}")
        case_ids.append(case_id)

    return case_ids

def apply_majority_vote(solutions):
    """
    Apply majority voting to predict solution - ENHANCED for drug crimes
    """
    # Count different solution types - ENHANCED for drug crimes
    punishment_types = [sol['jenis_hukuman'] for sol in solutions if sol['jenis_hukuman']]
    duration_types = [sol['durasi_hukuman'] for sol in solutions if sol['durasi_hukuman']]
    drug_types = [sol['jenis_narkoba'] for sol in solutions if sol['jenis_narkoba']]
    action_types = [sol['jenis_tindakan'] for sol in solutions if sol['jenis_tindakan']]

    # Get most common values
    most_common_punishment = Counter(punishment_types).most_common(1)
    most_common_duration = Counter(duration_types).most_common(1)
    most_common_drug = Counter(drug_types).most_common(1)
    most_common_action = Counter(action_types).most_common(1)

    majority_solution = {
        'metode': 'majority_vote',
        'jenis_hukuman': most_common_punishment[0][0] if most_common_punishment else 'tidak diketahui',
        'durasi_hukuman': most_common_duration[0][0] if most_common_duration else 'tidak diketahui',
        'jenis_narkoba': most_common_drug[0][0] if most_common_drug else 'tidak diketahui',
        'jenis_tindakan': most_common_action[0][0] if most_common_action else 'tidak diketahui',
        'confidence': len(punishment_types) / len(solutions) if solutions else 0
    }

    return majority_solution

def apply_weighted_similarity(query, case_ids, solutions):
    """
    Apply weighted similarity to predict solution - CORRECTED
    """
    # CORRECTED: Calculate weights based on similarity scores
    def clean_text_for_drug_cases(text):
        if pd.isna(text):
            return ""
        text = str(text).lower()
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        stopwords = ['dan', 'atau', 'yang', 'untuk', 'dari', 'dengan', 'pada',
                    'dalam', 'ke', 'di', 'adalah', 'oleh', 'akan', 'telah', 'sudah',
                    'ini', 'itu', 'tidak', 'ada', 'juga', 'dapat', 'bisa', 'harus',
                    'maka', 'saja', 'hanya', 'masih', 'sebuah', 'satu', 'dua',
                    'bahwa', 'tersebut', 'sebagai', 'atas', 'karena', 'sehingga']
        drug_terms = ['uud', 'kuhp', 'uu', 'pp', 'thc', 'mdma']
        words = text.split()
        words = [word for word in words if (len(word) > 2 or word in drug_terms) and word not in stopwords]
        return ' '.join(words)

    processed_query = clean_text_for_drug_cases(query)
    query_vector = tfidf_vectorizer.transform([processed_query])

    weights = []
    for case_id in case_ids:
        # Find case index by case_id
        case_row = df[df['case_id'] == case_id]
        if len(case_row) > 0:
            case_idx = case_row.index[0]
            case_vector = tfidf_matrix[case_idx:case_idx+1]
            similarity = cosine_similarity(query_vector, case_vector)[0][0]
            weights.append(similarity)
        else:
            weights.append(0.0)

    # Normalize weights
    total_weight = sum(weights)
    if total_weight > 0:
        weights = [w/total_weight for w in weights]
    else:
        weights = [1/len(weights)] * len(weights)

    # Weighted prediction - ENHANCED for drug crimes
    weighted_punishments = {}
    weighted_durations = {}
    weighted_drugs = {}
    weighted_actions = {}

    for i, (solution, weight) in enumerate(zip(solutions, weights)):
        punishment = solution.get('jenis_hukuman', '')
        duration = solution.get('durasi_hukuman', '')
        drug = solution.get('jenis_narkoba', '')
        action = solution.get('jenis_tindakan', '')

        if punishment:
            weighted_punishments[punishment] = weighted_punishments.get(punishment, 0) + weight
        if duration:
            weighted_durations[duration] = weighted_durations.get(duration, 0) + weight
        if drug:
            weighted_drugs[drug] = weighted_drugs.get(drug, 0) + weight
        if action:
            weighted_actions[action] = weighted_actions.get(action, 0) + weight

    # Get highest weighted predictions
    best_punishment = max(weighted_punishments.items(), key=lambda x: x[1]) if weighted_punishments else ('tidak diketahui', 0)
    best_duration = max(weighted_durations.items(), key=lambda x: x[1]) if weighted_durations else ('tidak diketahui', 0)
    best_drug = max(weighted_drugs.items(), key=lambda x: x[1]) if weighted_drugs else ('tidak diketahui', 0)
    best_action = max(weighted_actions.items(), key=lambda x: x[1]) if weighted_actions else ('tidak diketahui', 0)

    weighted_solution = {
        'metode': 'weighted_similarity',
        'jenis_hukuman': best_punishment[0],
        'durasi_hukuman': best_duration[0],
        'jenis_narkoba': best_drug[0],
        'jenis_tindakan': best_action[0],
        'confidence': (best_punishment[1] + best_duration[1] + best_drug[1] + best_action[1]) / 4
    }

    return weighted_solution

def combine_predictions(majority_solution, weighted_solution):
    """
    Combine different prediction methods - ENHANCED for drug crimes
    """
    # Simple combination: use weighted if confidence is higher
    if weighted_solution['confidence'] > majority_solution['confidence']:
        final_prediction = weighted_solution
        final_prediction['kombinasi'] = 'weighted_similarity_primary'
    else:
        final_prediction = majority_solution
        final_prediction['kombinasi'] = 'majority_vote_primary'

    # Add alternative prediction - ENHANCED for drug crimes
    final_prediction['alternatif'] = {
        'jenis_hukuman': weighted_solution['jenis_hukuman'] if final_prediction['metode'] == 'majority_vote' else majority_solution['jenis_hukuman'],
        'durasi_hukuman': weighted_solution['durasi_hukuman'] if final_prediction['metode'] == 'majority_vote' else majority_solution['durasi_hukuman'],
        'jenis_narkoba': weighted_solution['jenis_narkoba'] if final_prediction['metode'] == 'majority_vote' else majority_solution['jenis_narkoba'],
        'jenis_tindakan': weighted_solution['jenis_tindakan'] if final_prediction['metode'] == 'majority_vote' else majority_solution['jenis_tindakan']
    }

    return final_prediction

# =====================================================
# iii. IMPLEMENTASI FUNGSI
# =====================================================

# Pre-extract solutions from all cases
print("Extracting solutions from all drug crime cases...")
case_solutions = {}
for idx, row in df.iterrows():
    case_id = row['case_id']
    solution = extract_solution_from_case(row)
    case_solutions[case_id] = solution

print(f"✓ Extracted solutions from {len(case_solutions)} drug crime cases")

# =====================================================
# iv. DEMO MANUAL - CORRECTED FOR DRUG CRIMES
# =====================================================

def demo_solution_reuse():
    """
    Demonstrate solution reuse with 5 example drug crime cases
    """
    print("\n=== DEMO SOLUTION REUSE FOR DRUG CRIMES ===")

    # 5 example new drug crime cases
    demo_cases = [
        {
            'query': 'Terdakwa terbukti menjual narkoba jenis sabu-sabu seberat 2 gram kepada konsumen di Jakarta',
            'expected': 'pidana penjara sabu'
        },
        {
            'query': 'Pengedar ganja ditangkap polisi dengan barang bukti 500 gram ganja kering',
            'expected': 'pidana penjara ganja'
        },
        {
            'query': 'Terdakwa menggunakan narkotika heroin untuk keperluan pribadi',
            'expected': 'rehabilitasi atau pidana penjara'
        },
        {
            'query': 'Pelaku mengedarkan ekstasi di klub malam dengan keuntungan besar',
            'expected': 'pidana penjara ekstasi'
        },
        {
            'query': 'Bandar besar narkotika sabu-sabu dengan jaringan internasional',
            'expected': 'pidana penjara lama atau mati'
        }
    ]

    results = []

    for i, case in enumerate(demo_cases, 1):
        print(f"\n--- Demo Drug Crime Case {i} ---")
        print(f"Query: {case['query']}")
        print(f"Expected: {case['expected']}")

        # Get prediction
        prediction = predict_outcome(case['query'])

        print(f"Predicted Solution:")
        print(f"  - Metode: {prediction['metode']}")
        print(f"  - Jenis Narkoba: {prediction.get('jenis_narkoba', 'tidak diketahui')}")
        print(f"  - Jenis Tindakan: {prediction.get('jenis_tindakan', 'tidak diketahui')}")
        print(f"  - Jenis Hukuman: {prediction['jenis_hukuman']}")
        print(f"  - Durasi: {prediction['durasi_hukuman']}")
        print(f"  - Confidence: {prediction['confidence']:.3f}")
        print(f"  - Alternatif: {prediction['alternatif']}")

        # Get top similar cases for reference
        similar_cases = retrieve(case['query'], k=3)
        print(f"  - Top 3 Similar Cases: {similar_cases}")

        results.append({
            'query': case['query'],
            'prediction': prediction,
            'similar_cases': similar_cases
        })

    return results

# Run demo
demo_results = demo_solution_reuse()

# =====================================================
# v. OUTPUT - CORRECTED
# =====================================================

def create_prediction_script():
    """Create the required 04_predict.py script - CORRECTED"""
    script_content = '''
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity
import re
from collections import Counter

def predict_outcome(query: str) -> str:
    """
    Main prediction function as required for drug crime cases

    Args:
        query (str): New case description

    Returns:
        str: Predicted solution
    """
    # Load components - CORRECTED for drug crimes
    df = pd.read_csv('/content/drive/MyDrive/data/processed/drug_crime_cases_with_classification.csv')
    tfidf_vectorizer = joblib.load('/content/drive/MyDrive/models/drug_tfidf_vectorizer.pkl')
    tfidf_matrix = np.load('/content/drive/MyDrive/models/drug_tfidf_matrix.npy')

    # CORRECTED: Clean query using same method as case retrieval
    def clean_text_for_drug_cases(text):
        if pd.isna(text):
            return ""
        text = str(text).lower()
        text = re.sub(r'[^\\w\\s]', ' ', text)
        text = re.sub(r'\\s+', ' ', text)

        stopwords = ['dan', 'atau', 'yang', 'untuk', 'dari', 'dengan', 'pada',
                    'dalam', 'ke', 'di', 'adalah', 'oleh', 'akan', 'telah', 'sudah',
                    'ini', 'itu', 'tidak', 'ada', 'juga', 'dapat', 'bisa', 'harus',
                    'maka', 'saja', 'hanya', 'masih', 'sebuah', 'satu', 'dua',
                    'bahwa', 'tersebut', 'sebagai', 'atas', 'karena', 'sehingga']

        drug_terms = ['uud', 'kuhp', 'uu', 'pp', 'thc', 'mdma']
        words = text.split()
        words = [word for word in words if (len(word) > 2 or word in drug_terms) and word not in stopwords]
        return ' '.join(words)

    # Get similar cases
    processed_query = clean_text_for_drug_cases(query)
    query_vector = tfidf_vectorizer.transform([processed_query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_5_indices = similarities.argsort()[-5:][::-1]

    # Extract solutions from similar cases - ENHANCED for drug crimes
    solutions = []
    for idx in top_5_indices:
        case_data = df.iloc[idx]

        # Try multiple columns for punishment info
        punishment = None
        if pd.notna(case_data.get('jenis_hukuman')):
            punishment = str(case_data['jenis_hukuman'])
        elif pd.notna(case_data.get('status_putusan')):
            punishment = str(case_data['status_putusan'])

        if punishment:
            solutions.append(punishment)

    # Majority vote
    if solutions:
        most_common = Counter(solutions).most_common(1)[0][0]
        return most_common
    else:
        return "pidana penjara"  # Default prediction for drug crimes
'''

    # Save script
    script_path = '/content/drive/MyDrive/models/04_predict.py'
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(script_content)

    return script_path

# Create prediction script
script_path = create_prediction_script()

# Create evaluation results CSV - CORRECTED for drug crimes
def create_results_csv():
    """Create predictions.csv with results - CORRECTED for drug crimes"""
    results_data = []

    # Test queries for evaluation - CORRECTED for drug crimes
    test_queries = [
        "Terdakwa menjual narkoba sabu-sabu 2 gram",
        "Pengedar ganja 500 gram",
        "Pengguna heroin untuk pribadi",
        "Pengedar ekstasi di klub malam",
        "Bandar besar sabu-sabu jaringan internasional"
    ]

    for i, query in enumerate(test_queries, 1):
        prediction = predict_outcome(query)
        similar_cases = retrieve(query, k=5)

        results_data.append({
            'query_id': f'drug_query_{i:02d}',
            'predicted_solution': prediction['jenis_hukuman'] if isinstance(prediction, dict) else str(prediction),
            'predicted_drug_type': prediction.get('jenis_narkoba', 'tidak diketahui') if isinstance(prediction, dict) else 'tidak diketahui',
            'predicted_action': prediction.get('jenis_tindakan', 'tidak diketahui') if isinstance(prediction, dict) else 'tidak diketahui',
            'top_5_case_ids': ', '.join(similar_cases)
        })

    # Create DataFrame and save
    results_df = pd.DataFrame(results_data)
    results_path = '/content/drive/MyDrive/data/results/drug_crime_predictions.csv'

    # Create results directory
    os.makedirs('/content/drive/MyDrive/data/results', exist_ok=True)

    results_df.to_csv(results_path, index=False)
    print(f"✓ Results saved to: {results_path}")

    return results_df

# Create results
results_df = create_results_csv()

# Save solution database - CORRECTED for drug crimes
solution_db_path = '/content/drive/MyDrive/data/processed/drug_crime_solution_database.json'
with open(solution_db_path, 'w', encoding='utf-8') as f:
    json.dump(case_solutions, f, ensure_ascii=False, indent=2)

print(f"\n=== FINAL OUTPUT FOR DRUG CRIME SOLUTION REUSE ===")
print("✓ Script 04_predict.py created for drug crimes")
print("✓ predict_outcome() function implemented for drug crimes")
print("✓ Drug crime solution database created")
print("✓ Drug crime predictions CSV generated")

print(f"\n📁 File Structure:")
print("├── /content/drive/MyDrive/models/")
print("│   └── 04_predict.py")
print("├── /content/drive/MyDrive/data/processed/")
print("│   └── drug_crime_solution_database.json")
print("└── /content/drive/MyDrive/data/results/")
print("    └── drug_crime_predictions.csv")

print(f"\n📊 Results Preview:")
print(results_df.head())

print(f"\n🎯 Drug Crime Solution Reuse System Ready!")
print(f"   - {len(case_solutions)} drug crime solutions extracted")
print(f"   - Majority vote + Weighted similarity algorithms")
print(f"   - Enhanced for drug crime specific fields")
print(f"   - Ready for drug crime case prediction!")

# Display sample of results table
print(f"\n=== DRUG CRIME PREDICTIONS TABLE ===")
print("query_id | predicted_solution | predicted_drug_type | predicted_action | top_5_case_ids")
print("---------|-------------------|---------------------|------------------|---------------")
for _, row in results_df.iterrows():
    print(f"{row['query_id']} | {row['predicted_solution']} | {row['predicted_drug_type']} | {row['predicted_action']} | {row['top_5_case_ids']}")

✓ Loaded 64 drug crime cases
✓ Loaded TF-IDF vectorizer with 5000 features
Extracting solutions from all drug crime cases...
✓ Extracted solutions from 64 drug crime cases

=== DEMO SOLUTION REUSE FOR DRUG CRIMES ===

--- Demo Drug Crime Case 1 ---
Query: Terdakwa terbukti menjual narkoba jenis sabu-sabu seberat 2 gram kepada konsumen di Jakarta
Expected: pidana penjara sabu
Predicted Solution:
  - Metode: majority_vote
  - Jenis Narkoba: tidak diketahui
  - Jenis Tindakan: tidak diketahui
  - Jenis Hukuman: pidana penjara
  - Durasi: 4 tahun
  - Confidence: 1.000
  - Alternatif: {'jenis_hukuman': 'pidana penjara', 'durasi_hukuman': '4 tahun', 'jenis_narkoba': 'tidak diketahui', 'jenis_tindakan': 'tidak diketahui'}
  - Top 3 Similar Cases: ['drug_case_050', 'drug_case_005', 'drug_case_013']

--- Demo Drug Crime Case 2 ---
Query: Pengedar ganja ditangkap polisi dengan barang bukti 500 gram ganja kering
Expected: pidana penjara ganja
Predicted Solution:
  - Metode: majority_vote
  - Jeni